### Scrape data using Yelp APIs

In [ ]:
import requests
import json
import time
import io
from google.colab import files
import pandas as pd  
import config

# Testing API Endpoints

API_KEY = config.API_KEY
CLIENT_ID = config.CLIENT_ID

ENDPOINT = "https://api.yelp.com/v3/businesses/search"

HEADERS = {'Authorization': 'bearer %s' % API_KEY}

PARAMETERS = {'term': 'restaurants',
              'offset': 0,
              'limit': 50,
              'radius': 40000,
              'location': 'Toronto, ON'}

response = requests.get(url=ENDPOINT, params=PARAMETERS, headers=HEADERS)
restaurant_data = response.json()

uploaded = files.upload()
file_name = 'categories.json'
#io.StringIO[file_name].decode("utf-8")
data = json.loads(uploaded[file_name].decode("utf-8"))

# Load all categories
# each item in restaurant is dictionary
#with open(uploaded) as f:
#    data = json.load(uploaded)

restaurants = [place for place in data if 'restaurants' in place['parents']]

restaurant_aliases = [restaurant['alias'] for restaurant in restaurants]
restaurant_titles = [restaurant['title'] for restaurant in restaurants]

# Loop through all categories and collect data
restaurants_in_toronto = []

for category in restaurant_aliases:
    PARAMETERS['categories'] = category
    # Cycle through restaurants
    for offset_number in range(0,1000,50):
        PARAMETERS['offset'] = offset_number

        response = requests.get(url=ENDPOINT, params=PARAMETERS, headers=HEADERS)

        if not response.json().get('businesses', False):
            break

        restaurants_in_toronto.extend(response.json()['businesses'])

        print("{}: {}-{}".format(category, offset_number, offset_number+50))
        
        time.sleep(0.5) ## Don't want to get blocked by Yelp API

df_new = pd.io.json.json_normalize(restaurants_in_toronto)
print(df_new)

#restaurants_file =  open("toronto_restaurants_duplicates.json", "w")
#json.dump(restaurants_in_toronto, restaurants_file, indent=6)
#restaurants_file.close()

res_list = [value for counter, value in enumerate(restaurants_in_toronto) if value not in restaurants_in_toronto[counter+1:]]


#restaurants_file = open("toronto_restaurants.json", "w")
#json.dump(res_list, restaurants_file, indent=6)
#restaurants_file.close()

new_list = sorted(res_list,key=lambda k: k['name'])

#print(new_list)


Saving categories.json to categories (2).json
afghani: 0-50
african: 0-50
arabian: 0-50
argentine: 0-50
asianfusion: 0-50
asianfusion: 50-100
asianfusion: 100-150
australian: 0-50
austrian: 0-50
bangladeshi: 0-50
bbq: 0-50
bbq: 50-100
bbq: 100-150
belgian: 0-50
bistros: 0-50
brasseries: 0-50
brazilian: 0-50
breakfast_brunch: 0-50
breakfast_brunch: 50-100
breakfast_brunch: 100-150
breakfast_brunch: 150-200
breakfast_brunch: 200-250
breakfast_brunch: 250-300
breakfast_brunch: 300-350
breakfast_brunch: 350-400
british: 0-50
buffets: 0-50
burgers: 0-50
burgers: 50-100
burgers: 100-150
burgers: 150-200
burgers: 200-250
burgers: 250-300
burgers: 300-350
burgers: 350-400
burmese: 0-50
cafes: 0-50
cafes: 50-100
cafes: 100-150
cafes: 150-200
cafes: 200-250
cafes: 250-300
cafes: 300-350
cafes: 350-400
cajun: 0-50
cambodian: 0-50
caribbean: 0-50
caribbean: 50-100
caribbean: 100-150
cheesesteaks: 0-50
chicken_wings: 0-50
chicken_wings: 50-100
chicken_wings: 100-150
chicken_wings: 150-200
chicken_w

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


                          id  ...                           location.display_address
0     K3V9JWBZgIMtSICc4kBRUA  ...    [691 Yonge Street, Toronto, ON M4Y 2B2, Canada]
1     VDhLScy4FneNi9LcVOg-aQ  ...  [55 St Clair Avenue W, Toronto, ON M4V 2Y7, Ca...
2     GdCRQU3VCh_x8fY84mbwYQ  ...  [62 Overlea Boulevard, East York, ON M4H 1C4, ...
3     MvNRA9k-xF9ug5gXZexPog  ...      [62 Laird Drive, Toronto, ON M4G 3V1, Canada]
4     mKmriKe46YUIrTURfBDExg  ...         [119 Spadina Ave, Toronto, ON M5V, Canada]
...                      ...  ...                                                ...
9039  buMP3nR_-EL2ROE7nKMBUg  ...  [109 McCaul Street, Unit 28B, Toronto, ON M5T ...
9040  HPrYNgMkdQWEE4XLzLqPAA  ...  [469 Queen Street W, Toronto, ON M5V 2A9, Canada]
9041  e3zkVroknHTO7rX0RVr-Pg  ...        [94 York Blvd, Toronto, ON M7A 2C5, Canada]
9042  25IJ2tocJfC1ufW-qnRjMw  ...                          [Toronto, ON M5H, Canada]
9043  Ct3EA87rf_liNzwzbmNUgw  ...  [Dragon City Centre, 280 Spadi

In [ ]:
df_new.to_csv("May2021.csv")

### Assign neighbourhoods using postal codes

In [ ]:
# Assign to neighbourhood
import pandas as pd  

lookupPostal = {
    'M4R': 'Central Toronto (North Toronto West)',
    'M5K': 'Downtown Toronto (Toronto Dominion Centre / Design Exchange)',
    'M5S': 'Downtown Toronto (University of Toronto / Harbord)',
    'M3K': 'Other', # previously 'Downsview East (CFB Toronto)'
    'M4E': 'East Toronto (The Beaches)',
    'M4J': 'East Toronto (The Danforth East)',
    'M4M': 'East Toronto (Studio District)',
    'M4N': 'Central Toronto (Lawrence Park East)',
    'M4P': 'Central Toronto (Davisville North)',
    'M4S': 'Central Toronto (Davisville)',
    'M4W': 'Downtown Toronto (Rosedale)',
    'M4Y': 'Downtown Toronto (Church and Wellesley)',
    'M5B': 'Downtown Toronto (Ryerson)',
    'M5C': 'Downtown Toronto (St. James Park)',
    'M5E': 'Downtown Toronto (Berczy Park)',
    'M5G': 'Downtown Toronto (Central Bay Street)',
    'M5N': 'Central Toronto (Roselawn)',
    'M5X': 'Downtown Toronto (Underground city)',
    'M6G': 'Downtown Toronto (Christie)',
    'M6H': 'West Toronto (Dufferin / Dovercourt Village)',
    'M6R': 'West Toronto (Parkdale / Roncesvalles Village)',
    'M5J': 'Downtown Toronto (Harbourfront East / Union Station / Toronto Island)',
    'M6J': 'West Toronto (Rua Aþores / Trinity)',
    'M4K': 'East Toronto (The Danforth West / Riverdale)',
    'M4L': 'East Toronto (India Bazaar / The Beaches West)',
    'M4T': 'Central Toronto (Moore Park / Summerhill East)',
    'M4X': 'Downtown Toronto (St. James Town / Cabbagetown)',
    'M5L': 'Downtown Toronto (Commerce Court / Victoria Hotel)',
    'M5P': 'Central Toronto (Forest Hill North & West)',
    'M5R': 'Central Toronto (The Annex / North Midtown / Yorkville)',
    'M5T': 'Downtown Toronto (Kensington Market / Chinatown / Grange Park)',
    'M6P': 'West Toronto (High Park / The Junction South)',
    'M6S': 'West Toronto (Bloor West Village / Swansea)',
    'M8V': 'West Toronto (New Toronto / Mimico South / Humber Bay Shores)', # previously 'Etobicoke (New Toronto / Mimico South / Humber Bay Shores)'
    'M4V': 'Central Toronto (Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park)',
    'M5V': 'Downtown Toronto (CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / YTZ)',
    'M5W': 'Downtown Toronto Stn A PO Boxes 25 The Esplanade (Enclave of M5E)',
    'M7Y': 'East Toronto Business Reply Mail Processing Centre 969 Eastern (Enclave of M4L)',
    'M5A': 'Downtown Toronto (Regent Park / Port of Toronto)',
}

with open ('May2021.csv', 'r') as myfile:
  MayNov = pd.read_csv(myfile)
 
labelNeigh = []
code = []

print(MayNov.shape)

isNaNCode = MayNov['location.zip_code'] == MayNov['location.zip_code']
MayNov = MayNov[isNaNCode]

print(MayNov.shape)

for index, row in MayNov.iterrows():
  code.append(row['location.zip_code'])
    
for i in code:
  if i[:3] in lookupPostal:
    labelNeigh.append(lookupPostal[i[:3]])
  else:
    labelNeigh.append('Other')


(9044, 25)
(9016, 25)


In [ ]:
print(MayNov.shape)
MayNov.insert(0, "Specific Neighbourhood", labelNeigh, True) 
print(MayNov.shape)

(9016, 25)
(9016, 26)


In [ ]:
nei = []

for index, row in MayNov.iterrows():
  string = row['Specific Neighbourhood']
  if string == 'Other':
    nei.append('Other')
  else:
    result = string[:string.index("(")]
    nei.append(result)

MayNov.insert(0, "Neighbourhood", nei, True) 
print(MayNov.shape)

(9016, 27)


### Assess openings and closures

In [ ]:
May2020 = pd.read_excel('TO_rest_May2Nov_cleaned.xlsx')
May2021 = pd.read_excel('May2021.xlsx')

#print(May2020.shape)
#print(May2021.shape)
print(May2020.columns.to_list())
print(May2021.columns.to_list())

df1 = May2020[['id', 'name_x']]
df2 = May2021[['id', 'name']]
print("size of cleaned dataset: "+ str(df1.shape))
print("size of newly aggregated dataset: "+ str(df2.shape))
df1 = pd.DataFrame(df1)
df2 = pd.DataFrame(df2)
df1.rename(columns={'name_x':'name'}, inplace=True)

# 532 restaurants only in 2020
only_in_2020 = df1.merge(df2, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
# 666 restaurants only in 2021
only_in_2021 = df1.merge(df2, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']
# occurs in both datastes
across_2020_2021 = df1.merge(df2, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='both']

print("closed: " + str(only_in_2020.shape))
print("new: " + str(only_in_2021.shape))

# dropping ALL duplicte values
df2.drop_duplicates(subset ="id",
                     keep = "first", inplace = True)
print("After dropping duplicates: " + str(df2.shape)) #~60% of restaurants only have 1 category and the rest have 2+ categories 

['Column1', 'id', 'Status', 'name_x', 'is_closed_x', 'review_count_x', 'Categories', 'Categority', 'categories_x', 'rating_x', 'phone_x', 'distance_x', 'coordinates.latitude_x', 'coordinates.longitude_x', 'location.address1_x', 'location.address2_x', 'location.address3_x', 'location.city_x', 'location.zip_code_x', 'location.country_x', 'location.state_x', 'Neighbourhood', 'Specific Neighbourhood']
['Unnamed: 0', 'Neighbourhood', 'Specific Neighbourhood', 'Unnamed: 0.1', 'id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'transactions', 'price', 'phone', 'display_phone', 'distance', 'coordinates.latitude', 'coordinates.longitude', 'location.address1', 'location.address2', 'location.address3', 'location.city', 'location.zip_code', 'location.country', 'location.state', 'location.display_address']
size of cleaned dataset: (5978, 2)
size of newly aggregated dataset: (9016, 2)
closed: (532, 3)
new: (666, 3)
After dropping duplicates: (5836, 2)


In [ ]:
closed_dict = set(only_in_2020['id'])
new_dict = set(only_in_2021['id'])

In [ ]:
# iterate through 2020 dataset and clasify them as closed and open
openClose = []

for index, row in May2020.iterrows():
  if row['id'] in closed_dict:
    openClose.append('Closed')
  else:
    openClose.append('Open')

May2020.insert(0, "2021 Assessment", openClose, True)

May2020.to_excel('Yelp2020_Reassessment.xlsx')

# iterate through 2021 dataset and isolate the "new" restaurants
# keep the first appearing category (alphabetical order)
May2021.drop_duplicates(subset ="id",
                     keep = "first", inplace = True)
newOld = []

for index, row in May2021.iterrows():
  if row['id'] in new_dict:
    newOld.append('New')
  else:
    newOld.append('Old')

May2021.insert(0, "2021 Assessment", newOld, True)

May2021.to_excel('Yelp2021_Reassessment.xlsx')

print(May2021.shape)


(5836, 30)
